# データサイエンス100本ノック（構造化データ加工編） - Python

## はじめに
- 初めに以下のセルを実行してください
- 必要なライブラリのインポートとデータベース（PostgreSQL）からのデータ読み込みを行います
- pandas等、利用が想定されるライブラリは以下セルでインポートしています
- その他利用したいライブラリがあれば適宜インストールしてください（"!pip install ライブラリ名"でインストールも可能）
- 処理は複数回に分けても構いません
- 名前、住所等はダミーデータであり、実在するものではありません

In [1]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, date
from dateutil.relativedelta import relativedelta
import math
import psycopg2
from sqlalchemy import create_engine
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from imblearn.under_sampling import RandomUnderSampler # conda install -c conda-forge imbalanced-learn

df_customer = pd.read_csv("./data/customer.csv")
df_category = pd.read_csv("./data/category.csv")
df_product = pd.read_csv("./data/product.csv")
df_receipt = pd.read_csv("./data/receipt.csv")
df_store = pd.read_csv("./data/store.csv")
df_geocode = pd.read_csv("./data/geocode.csv")

C:\Users\iiats\Anaconda3\envs\Python36-gpu\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# 演習問題

---
> P-031: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の標本標準偏差を計算し、降順でTOP5を表示せよ。

In [2]:
# Week-06 で鍛えられたのですんなり解けました。
# Pandas で標準偏差を出す方法： https://it-engineer-lab.com/archives/1065?fbclid=IwAR29InKEJcB6eeF6TL56xEVFfCsXHQodFSK5K_sPFAWHL-E-bQy2dWU12kA
df_receipt2 = df_receipt.groupby('store_cd').amount.std(ddof=0).reset_index()
df_receipt3 = df_receipt2.sort_values('amount', ascending=False)
df_receipt3.head()

,store_cd,amount
28,S13052,663.391816
31,S14011,553.456916
42,S14034,544.903736
5,S13001,543.536561
12,S13015,543.409938


---
> P-032: レシート明細データフレーム（df_receipt）の売上金額（amount）について、25％刻みでパーセンタイル値を求めよ。

In [3]:
# パーセンタイル値ってなんだ？　https://bellcurve.jp/statistics/glossary/1388.html
# Pandas でパーセンタイルを取得する方法　https://note.nkmk.me/python-pandas-quantile/
perc = [0, 0.25, 0.5, 0.75, 1]
df_receipt.amount.quantile(perc)

0.00       10.0
0.25      102.0
0.50      170.0
0.75      288.0
1.00    10925.0
Name: amount, dtype: float64

In [4]:
# Numpy でパーセンタイルを取得する方法　https://analytics-note.xyz/programming/numpy-percentile/
amount = df_receipt.amount
perc = [25, 50, 75, 100] # 0 を入れるとエラー
np.percentile(amount, perc)

array([  102.,   170.,   288., 10925.])

---
> P-033: レシート明細データフレーム（df_receipt）に対し、店舗コード（store_cd）ごとに売上金額（amount）の平均を計算し、330以上のものを抽出せよ。

In [5]:
# 割とストレートな問題です
df_receipt2 = df_receipt.groupby('store_cd').amount.mean().reset_index()
df_receipt2.query('amount >= 330')

,store_cd,amount
1,S12013,330.194130
5,S13001,348.470386
7,S13003,350.915519
8,S13004,330.943949
12,S13015,351.111960
16,S13019,330.208616
17,S13020,337.879932
28,S13052,402.867470
30,S14010,348.791262
31,S14011,335.718333


---
> P-034: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求めよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。


In [6]:
# 顧客IDが"Z"から始まるのものは非会員を表すため除外する方法がわからなくて公式解答をカンニング
df_receipt2 = df_receipt[~df_receipt['customer_id'].str.startswith("Z")]
df_receipt2

# ~ はビット反転演算子
# Pandas では~ はNOTの意味 https://note.nkmk.me/python-pandas-multiple-conditions/

,sales_ymd,sales_epoch,store_cd,receipt_no,receipt_sub_no,customer_id,product_cd,quantity,amount
0,20181103,1257206400,S14006,112,1,CS006214000001,P070305012,1,158
1,20181118,1258502400,S13008,1132,2,CS008415000097,P070701017,1,81
2,20170712,1215820800,S14028,1102,1,CS028414000014,P060101005,1,170
4,20180821,1250812800,S14025,1102,2,CS025415000050,P060102007,1,90
5,20190605,1275696000,S13003,1112,1,CS003515000195,P050102002,1,138
...,...,...,...,...,...,...,...,...,...
104671,20180131,1233360000,S14010,1102,1,CS010414000008,P060103003,1,150
104673,20181217,1261008000,S13004,1142,2,CS004515000066,P059001016,1,308
104674,20190911,1284163200,S14046,1182,1,CS046415000017,P070703003,1,98
104678,20170311,1205193600,S14040,1122,1,CS040513000195,P050405003,1,168


In [7]:
df_receipt2.groupby('customer_id').amount.sum().mean()

2547.742234529256

In [8]:
# これも公式解答の答え
# queryを使う書き方
df_receipt.query('not customer_id.str.startswith("Z")', engine='python').groupby('customer_id').amount.sum().mean()

# query() 最強！

2547.742234529256

---
> P-035: レシート明細データフレーム（df_receipt）に対し、顧客ID（customer_id）ごとに売上金額（amount）を合計して全顧客の平均を求め、平均以上に買い物をしている顧客を抽出せよ。ただし、顧客IDが"Z"から始まるのものは非会員を表すため、除外して計算すること。なお、データは10件だけ表示させれば良い。

In [9]:
amount = df_receipt[~df_receipt['customer_id'].str.startswith("Z")].groupby('customer_id').amount
mean = amount.sum().mean()
mean

2547.742234529256

In [10]:
df_receipt2 = df_receipt.groupby('customer_id').amount.sum().reset_index()
df_receipt2.head()

,customer_id,amount
0,CS001113000004,1298
1,CS001114000005,626
2,CS001115000010,3044
3,CS001205000004,1988
4,CS001205000006,3337


In [11]:
df_receipt3 = df_receipt2[df_receipt2.amount >= mean]
df_receipt3.head(10)

,customer_id,amount
2,CS001115000010,3044
4,CS001205000006,3337
13,CS001214000009,4685
14,CS001214000017,4132
17,CS001214000052,5639
21,CS001215000040,3496
30,CS001304000006,3726
32,CS001305000005,3485
33,CS001305000011,4370
53,CS001315000180,3300
